In [1]:
import pandas as pd 
from pathlib import Path
import os

In [2]:
cd ..

c:\Gan\GAN-market-prediction


In [10]:
stock_path = Path(os.path.abspath("")) / "data" / "stock_data"
save_path = Path(os.path.abspath("")) / "publication_plots"
tickers = ["EA", "UBSFY", "TTWO", "ATVI"]

In [6]:
df = pd.DataFrame()
for ticker in tickers:
    tmp = pd.read_csv(stock_path / f"{ticker}.csv")
    tmp["Company"] = ticker
    df = df.append(tmp)

C:\Users\kuba1\AppData\Local\Temp\ipykernel_10724\1575495433.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_10724\1575495433.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_10724\1575495433.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_10724\1575495433.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)


In [9]:
df.groupby(by="Company").describe().T

Company                     ATVI            EA          TTWO         UBSFY
Open         count  7.570000e+02  7.570000e+02  7.570000e+02  7.570000e+02
             mean   6.837792e+01  1.174786e+02  1.434314e+02  1.535412e+01
             std    1.811106e+01  2.062370e+01  3.220382e+01  2.450012e+00
             min    3.956548e+01  7.472812e+01  8.471000e+01  9.050000e+00
             25%    5.210339e+01  9.717636e+01  1.172500e+02  1.393000e+01
             50%    6.857000e+01  1.193464e+02  1.383700e+02  1.548000e+01
             75%    8.241098e+01  1.385884e+02  1.711000e+02  1.681500e+01
             max    1.033197e+02  1.485431e+02  2.104765e+02  2.089000e+01
High         count  7.570000e+02  7.570000e+02  7.570000e+02  7.570000e+02
             mean   6.922366e+01  1.189510e+02  1.453735e+02  1.546871e+01
             std    1.817212e+01  2.062690e+01  3.241062e+01  2.460111e+00
             min    4.119361e+01  7.691432e+01  8.757000e+01  9.270000e+00
             25%    5.284490e+01  9.860734e+01  1.190800e+02  1.396000e+01
             50%    7.004000e+01  1.206978e+02  1.407800e+02  1.564000e+01
             75%    8.327678e+01  1.402643e+02  1.728500e+02  1.694000e+01
             max    1.040263e+02  1.495559e+02  2.149100e+02  2.134000e+01
Low          count  7.570000e+02  7.570000e+02  7.570000e+02  7.570000e+02
             mean   6.738189e+01  1.158599e+02  1.413383e+02  1.521634e+01
             std    1.794660e+01  2.052602e+01  3.182764e+01  2.440169e+00
             min    3.908489e+01  7.344622e+01  8.441000e+01  9.050000e+00
             25%    5.142120e+01  9.571559e+01  1.158500e+02  1.374000e+01
             50%    6.707523e+01  1.170509e+02  1.363400e+02  1.538000e+01
             75%    8.128642e+01  1.370253e+02  1.689600e+02  1.667000e+01
             max    1.020559e+02  1.457801e+02  2.094350e+02  2.067000e+01
Close        count  7.570000e+02  7.570000e+02  7.570000e+02  7.570000e+02
             mean   6.832515e+01  1.174449e+02  1.434253e+02  1.533973e+01
             std    1.802681e+01  2.056176e+01  3.208114e+01  2.456598e+00
             min    3.933990e+01  7.425114e+01  8.463000e+01  9.140000e+00
             25%    5.210339e+01  9.698756e+01  1.176000e+02  1.384010e+01
             50%    6.947362e+01  1.190582e+02  1.381900e+02  1.551000e+01
             75%    8.230000e+01  1.384656e+02  1.706700e+02  1.682000e+01
             max    1.033098e+02  1.482325e+02  2.133400e+02  2.124000e+01
Volume       count  7.570000e+02  7.570000e+02  7.570000e+02  7.570000e+02
             mean   7.852192e+06  3.352595e+06  1.682517e+06  1.603439e+05
             std    4.732384e+06  2.796399e+06  1.258285e+06  3.360945e+05
             min    1.562888e+06  6.060640e+05  2.116420e+05  1.188800e+04
             25%    5.243074e+06  2.068342e+06  1.023114e+06  3.837000e+04
             50%    6.692846e+06  2.723601e+06  1.351017e+06  5.767100e+04
             75%    8.966312e+06  3.757614e+06  1.913890e+06  1.117520e+05
             max    5.170888e+07  3.870450e+07  1.894501e+07  3.997347e+06
Dividends    count  7.570000e+02  7.570000e+02  7.570000e+02  7.570000e+02
             mean   1.651255e-03  1.122853e-03  0.000000e+00  0.000000e+00
             std    2.632308e-02  1.377952e-02  0.000000e+00  0.000000e+00
             min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
             25%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
             50%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
             75%    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
             max    4.700000e-01  1.700000e-01  0.000000e+00  0.000000e+00
Stock Splits count  7.570000e+02  7.570000e+02  7.570000e+02  7.570000e+02
             mean   0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
             std    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
             min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
             25%    0.000

In [13]:
with open(save_path / "desc_stock.tex", "w") as f:
    f.write(
        df.loc[
            :, [col for col in df.columns if col not in ["Dividends", "Stock Splits"]]
        ]
        .groupby(by="Company")
        .describe()
        .T.to_latex(caption="Descriptive statistics for chosen companies")
    )


C:\Users\kuba1\AppData\Local\Temp\ipykernel_10724\838483005.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.loc[


In [3]:
results_dict = {
    "ATVI": [1.3585, 1.8867],
    "EA": [1.3427, 1.8956], 
    "TTWO": [3.5405, 4.5069],
    "UBSFY": [0.2578, 0.3376]
}

In [6]:
pd.DataFrame(results_dict).rename(index={0: "MAE", 1:"RMSE"}).to_latex(caption="Error metrics for chosen companies")

,ATVI,EA,TTWO,UBSFY
MAE,1.3585,1.3427,3.5405,0.2578
RMSE,1.8867,1.8956,4.5069,0.3376
